# 1. Обработка данных и создание первых датасетов

Для первого этапа исследования пренебрежём структурой данных. Проигнорируем то, что у нас есть несколько разных кварталов, в течение которых людям ставятся оценки. Посмотрим на рспределение Target_K_ocenki. Переменная принимает значения от 0 до 7. Значение больше двух приниемает 4 наблюдения. Откидываем эти аномалии. Оценку меньше 1 получили 2% людей. Ими мы тоже пренебрежём. При построении модели будем смотреть только на отрезок от 1 до 2. Сразу же отметим, что у нас в выборке очень много людей, получивших оценку 1.

![ ](https://pp.userapi.com/c840239/v840239510/29ee0/KSY1p1ZqBnw.jpg)

Посмотрим на то сколько у нас есть пропусков в объясняющих переменных. В данных есть две группы переменных с очень большим количеством пропусков (на графике пропуски обозначены жёлтым). Это переменные класса Talent_Q и переменные, связанные с количеством отправленных писем.  

<img align="center" src="https://pp.userapi.com/c840239/v840239510/29ee9/zFSFObEmC_Y.jpg" height="700" width="700"> 

Взглянем на количество пропусков в каждой переменной. Наверное, не очень много смысла включать в данные переменные с пропусками более чем в 50 тысячах наблюдений из 70 тысяч. 

* Без пропусков: 9 переменных
* Больше 40k пропусков: 139 переменных
* Между 20k и 40k пропусков: 15 переменных
* Меньше 20k пропусков: 159 переменных

<img align="center" src="https://pp.userapi.com/c840239/v840239510/29efc/jRq_2tzfYdE.jpg" height="500" width="500">

Чуть более подробно проанализируем структуру наблюдений по кварталам. Количество наблюдений от квартала к кварталу колеблется в диапозоне от 4000 (начало наблюдений) до 7000 (конец наблюдений) и постоянно увеличивается. Отложим по оси $x$ кварталы, а по оси $y$ частоту сотрудников, у которых стоит в оценка больше 1 на первом графике и стоит оценка равная 1 на втором. 

<img align="center" src="https://sun9-9.userapi.com/c840631/v840631425/c0c6/w1xxRsM885I.jpg" height="700" width="700"> 

Во втором квартале 2015 года явно произошёл какой-то струтурный сдвиг, который привёл к ужесточению оценивания сотрудников. Запомним этот занимательный факт.

Разобьём все фичи на бинарные и категориальные. После сформируем несколько датасетов.

**Инструкция по обработке данных: **
1. Выкидываем пропуски по какому-то правилу. 
2. Разделяем датасет на реальные признаки и категориальные. 
3. Заполняем по какому-то правилу пропуски в реальных признаках (среднее, медиана, ноль, регрессия).
4. Отнормируем реальные переменные. 
5. Выделим все пропуски в категориальных переменных в отдельный класс. 
6. Сделаем WOE-преобразование по следующему принципу:  разделим непрерывную переменную на два равных блока. Если она принимает значение 1, то закодируем этот факт нулём. Если она принимает значение больше 1, закодируем этот факт единицей. Эту бинарную переменную сохраним и запомним как отдельную целевую переменную. С помощью неё и сделаем WOE-преобразование. 
7. Сливаем категориальные и реальные переменные в единое целое. 

Вместо WOE-преобразования можно сделать OHE-преобразование. Оценивание показало, что выбор метода работы с категориальной переменной никак не сказывается на качество моделей на этом датасете.

По такому алгоритму сформируем несколько разных датасетов: 

1. Данные X l20 drna (меньше 20k пропусков в переменных, все оставшиеся наблюдения с пропусками удалены). Размерность: $(44437, 159)$.

2. Данные X g20 l40 med   (все признаки с более чем 40k пропусками удалены, остальные для реальных переменных заполнены медианами). Размерность: $(70132, 174)$.

3. Данные X g20 l40 zero  (все признаки с более чем 40k пропусками удалены, остальные для реальных переменных заполнены нулями). Размерность: $(70132, 174)$. Результаты на этой выборке ничем не отличались от медианных. Было решено её не использовать.

4. Данные X g20 l50 med   (все признаки с более, чем 50k пропусками удалены, остальные для реальных переменных заполнены медианами. Размерность: $(70132, 240)$.

5. Возьмём данные, которыми мы располагаем после структурного сдвига. Возможно, более старые данные будут искажать картину и портить модель. Отсеем всё, где более 35k пропусков. Остальные реальные пропуски заполним медианами. Размерность: $(50246, 261)$.

Поделим датасеты на тренировочную и тестовую выборки в пропорции 75 к 25. 

# 2. Регрессия.

Будем действовать по следующей методологии:

1. Строим несколько константных прогнозов в качестве baseline. 
2. Выращиваем случайный лес. Обычно случайный лес работает очень хорошо и, если в данных есть хоть какие-то закономерности, он их находит. 
3. Если лес нашёл какие-то нелинейные взаимосвязи, пытаемся отобрать самые важные фичи по следующим четырём стратегиям и воспитать на них полином. 
    * Стратегия 1:  Отбор признаков по корреляции Спирмана. 
    * Стратегия 2:  Отбор признаков с помощью деревьев.
    * Стратегия 3:  Отбор признаков с помощью $R^2$.
    * Стратегия 4:  Отбор признаков с помощьб Lasso-регрессии. 

Подробно опишем все манипуляции, которые были сделаны на втором датасете. После опишем особенности, которые были получены на остальных датасетах. 

### Второй датасет


| Модель               |  MSE трэйн | MSE тест  | 
|---------------------:|------------|-----------|
| Прогноз по среднему  | 0.041      |           |
| Прогноз по медиане   | 0.046      |           |
| Лес из 10 деревьев   | 0.006      | 0.036     |
| Линейная регрессия   | 0.038      | 0.038     | 
| Lasso-регрессия      | 0.038      | 0.038     |
| Полином 1            | 0.039      | 0.039     | 
| Полином 2            | 0.040      | 0.039     | 
| Полином 3            | 0.039      | 0.039     | 
| Полином 4            | 0.039      | 0.038     | 


Оценка качества по лесу существенно отличается от baseline. Построим визуализацию, в которой по оси абсцисс отложим значения $y$, а по оси ординат $\hat{y}$. Очевидно, что чем сильнее вытянуты точки вдоль диагонали на тестовой выборке, тем лучше будет качество у модели.  

<img align="center" src="https://pp.userapi.com/c837635/v837635021/5edd3/4Z5RPhnbodM.jpg" height="700" width="700">

Оценка качества по обычной линейной регрессии со всеми признаами уступает качеству по лесу. Качество на трейне незначительно отличается от качества на тесте. Судя по всему модель не переобучается. Тем не менее, всё-равно сделаем перебор по рештке и оценим Lasso-регерссию.

<img align="center" src="https://pp.userapi.com/c837635/v837635021/5eddd/1AW3YX9XNCk.jpg" height="700" width="700">

Попробуем заняться отбором признаков. Провернём все четыре стратегии и выведем топ из самых важных признаков и их "важность". 

| Коэффициент Спирмана     |      | $R^2$                 |          | Деревья               |          | Lasso                   |          |
|--------------------------|------|-----------------------|----------|-----------------------|----------|-------------------------|----------|
| K_rab_time               | 0.14 | K_rab_time            | 1.56e-02 | Age                   | 1.76e-02 | Age                     | 1.76e-02 |
| Grade                    | 0.09 | K_otrab_time          | 1.49e-02 | Grade                 | 1.70e-02 | K_otrab_time            | 1.41e-02 |
| Age                      | 0.08 | Age                   | 8.32e-03 | K_otrab_time          | 1.41e-02 | Sluzheb_komand          | 5.12e-03 |
| K_otrab_time             | 0.08 | Sluzheb_komand_binar  | 5.14e-03 | Oklad_relative        | 1.34e-02 | aTimeNet_WD             | 6.38e-03 |
| Sluzheb_komand_binar     | 0.08 | Experience_general    | 5.04e-03 | K_rab_time            | 1.31e-02 | Oklad_relative          | 1.34e-02 |
| Sluzheb_komand           | 0.07 | years_after_vuz       | 4.87e-03 | Experience_general    | 1.16e-02 | aTimeGross_WD_avg       | 4.67e-03 |
| prd_da_active_nflag      | 0.07 | Oklad_relative        | 4.57e-03 | Grade_avg             | 1.01e-02 | Grade                   | 1.70e-02 |
| Netrud_rabotnik_binar    | 0.07 | Netrud_rabotnik_binar | 3.68e-03 | Experience_continuous | 9.90e-03 | Experience_continuous   | 9.90e-03 |
| tour_to_spend_prcntl_pct | 0.07 | Netrud_rabotnik       | 3.49e-03 | Experience_SB         | 9.89e-03 | Grade_avg               | 1.01e-02 |
| Cnt_total_empl_manag_new | 0.06 | Age_avg               | 2.85e-03 | years_after_vuz       | 9.86e-03 | bty_to_spend_prcntl_pct | 6.19e-03 |


Посмотрим как важность признаков выглядит на картинке, по оси абцисс, на которой будут отложены переменные, а по оси ордина "важность признака". Для этого отшкалируем важность в отрезок $[0;1]$. 

<img align="center" src="https://pp.userapi.com/c639130/v639130028/47c9d/KFiV2IOJAg4.jpg
" height="1000" width="1000">


Возьмём по 20 самых важных переменных из каждого критерия и оценим полиномы степени два. Отметим, что качество этих полиномов незначительно отличается от качества линейной регрессии. Lasso-регрессия зануляет огромное число признаков. Обычно пересечение по всем 80 отобранным признакам даёт около 50 уникальных значений. В какой-то пропрорции методы отбирают одинаковые переменные. 

Если взять лучшие 20 признаков по каждому методу и после взять пересечение по ним, получим множество из 48 признаков. Осстальные 32 повторялись. 

### Другие датасеты 

Качество ошибки на других датасетах ведёт себя аналогично... 


# 2. Деревья, леса, бустинг

Лес из 10 деревьев уже побил по своему качеству регрессию. Попробуем построить одно дерево и посмотреть как оно будет работать на наших данных. 

<img align="center" src="https://pp.userapi.com/c841229/v841229028/20216/27DL-K_ylTs.jpg" height="700" width="700">

Модель явно переобучилась. Обрезка листьев повышает качество модели, но приводит не к оптимальному результату. Для построения оптимального дерева создадим решётку из параметров и сделаем по ней перебор. 

<img align="center" src="https://pp.userapi.com/c841229/v841229028/20232/ndqzForC8mc.jpg" height="700" width="700">

Построем оптимальный (используя дерево с оптимальными параметрами из предыдущего пугкта) и неоптимальный (дерево не специфицировано) леса из 1000 деревьев. Оптимальный лес показывает одинаковое качество на трэйне и тесте в районе 0.38 на всех выборках. Неоптимальный лес показывает качество в районе 0.32 на тесте, но при этом запоминает весь трэйн. 

<img align="center" src="https://pp.userapi.com/c837334/v837334503/4eaab/G4Nz9SyGxEc.jpg
" height="700" width="700">

Попробуем взять лучшие признаки, отобранные в предыдущем скрипте и посмотреть насколько сильно изменится качество леса. Если использовать неспецифицированное дерево, качество на трэйне заметно выше, чем на тесте. Использование нашего самого лучшего дерева позволяет это качество сравнять и немного улучшить работу алгоритма. За компанию прогоним XGBoost. 

<img align="center" src="https://pp.userapi.com/c841524/v841524209/249a3/q0Ps0uvX5Vk.jpg" height="700" width="700">



| Модель               |  MSE трэйн | MSE тест  | 
|---------------------:|------------|-----------|
| Прогноз по среднему  | 0.041      |           |
| Прогноз по медиане   | 0.044      |           |
| Линейная регрессия   | 0.039      | 0.038     | 
| Дерево               | 0.000      | 0.066     | 
| Оптимальное Дерево   | 0.037      | 0.038     |
| Лес из 1000 деревьев | 0.005      | 0.032     |
| Оптимальный лес(1000)| 0.038      | 0.038     |
| Обычный 1            | 0.007      | 0.036     | 
| Обычный 2            | 0.006      | 0.035     | 
| Обычный 3            | 0.007      | 0.039     | 
| Обычный 4            | 0.007      | 0.037     | 
| Оптимальный 1        | 0.038      | 0.037     | 
| Оптимальный 2        | 0.038      | 0.038     | 
| Оптимальный 3        | 0.038      | 0.038     | 
| Оптимальный 4        | 0.039      | 0.039     | 
| XGBoost              | 0.036      | 0.036     |




# 3. Ближайшие соседи




Метод ближайших соседей ничем также не порадовал. Значительного улучшения в качестве получено не было. 

<img align="center" src="https://pp.userapi.com/c841534/v841534209/20532/lh0YHhXTOaI.jpg" height="700" width="700">

Была предпринята попытка обучить метод ближайших соседей только на лучших признаках. В этой ситуации также значимого улучшения достигнуто не было. Качество метода на 20 лучших признаках по каждой стратегии отбора оказалось чуть более плохим, чем на полной выборке. Мы откинули много важной информации. 

| Модель               |  MSE трэйн | MSE тест  | 
|---------------------:|------------|-----------|
| Прогноз по среднему  | 0.041      |           |
| Прогноз по медиане   | 0.044      |           |
| KNN                  |            |  0.038    |
| KNN  Спирман         |            |  0.045    |
| KNN   R^2            |            |  0.044    |
| KNN   Деревья        |            |  0.040    |
| KNN   Lasso          |            |  0.044    |


# Сводная таблица из всех моделей для всех датасетов (качество на тесте)



| Модель                      | X g20 l40 med| X g20 l50 med| Xl20 drna | X sdvig l35 med| 
|----------------------------:|--------------|--------------|-----------|----------------|
| Прогноз по среднему         | 0.041        | 0.041        |   0.042   |   0.041        |
| Прогноз по медиане          | 0.044        | 0.046        |   0.043   |   0.053        |
| Лес из 10 деревьев          | 0.036        | 0.035        |   0.038   |   0.036        |
| Линейная регрессия          | 0.039        | 0.038        |   0.048   |   0.041        | 
| Lasso-регрессия             | 0.039        | 0.038        |   0.041   |   0.038        |
| Лучший Полином 2 степ       | 0.039        | 0.038        |   0.040   |   0.038        | 
| Дерево                      | 0.067        | 0.066        |   0.067   |   0.067        | 
| Оптимальное Дерево          | 0.037        | 0.038        |   0.041   |   0.038        |
| Лес из 1000 деревьев        | 0.032        | 0.032        |   0.034   |   0.033        |
| Оптимальный лес(1000)       | 0.038        | 0.038        |   0.040   |   0.038        |
| Лучший лес с отбором        | 0.036        | 0.036        |   0.037   |   0.036        | 
| Оптимальный лес с отбором   | 0.038        | 0.038        |   0.040   |   0.038        |
| XGBoost                     | 0.036        | 0.036        |   0.038   |   0.036        |
| KNN                         | 0.038        |    -         |    -      |     -          |



# 4. Бинаризация целевой переменной.

Нам удалось добиться на изначальной выборке некоторого улучшения по сравнению с константным прогнозом, но нам этого мало. Как уже говорилось выше, в выборке довольно много единичек. Интересно было бы попробовать сначала идентифицировать является ли человек единичкой, а после, если он получает по нашему прогнозу оценку выше единицы, пытаться узнать насколько высокую оценку он получит.

Перед тем как строить такую двуступенчатую модель имеет смысл попробовать работать чисто с бинарной переменной и попытаться с приемлимым качеством найти все единицы. Бинаризуем переменную по следующему принципу: целевая переменная равна нулю, если непрерывная равна единице и бинарная равна единице, если целевая больше единицы. Выборка будес сбалансированна. Около 50% людей получают в качестве своей оценки единицу.  

Работать снова будем со второй выборкой. Все результаты, которые будут из неё выбиваться, мы опишем отдельно. 

### Логистическая регрессия 

Бинаризация переменной и обучение логистической регрессии сразу же увеличивает Roc-auc До 67%. Судя по всему, часть единичек удастся идентифицировать довольно хорошо. Попробуем добиться улучшения и оценить более сложные модели с бинарной переменной. 

Лассо-логистическая регрессия с оптимальным, подобранным по сетке, гиперпараметром не приводит к значительному улучшению модели. Кривая зависимости коэфициента регуляризации от Log-loss выглядит следующим образом (по оси абсцисс значение параметра, по оси ординат логлосс).  

<img align="center" src="https://pp.userapi.com/c837334/v837334503/4eb03/Dpc5K0QjgBg.jpg" height="500" width="500">

Для оптимальной логистической регрессии Roc-кривая выглядит следующим образом: 

<img align="center" src="https://pp.userapi.com/c841521/v841521496/20d3d/HpXzEBVzkh8.jpg" height="500" width="500">

### Деревья 

Базовое дерево и дерево с подбором параметров по решотке дают доаольно хороший результат. Тем не менее, они не побили логистическую регерссию. Тем не менее, случайный леса поднимают значение toc-auc до 0.7. 

<img align="center" src="https://sun9-9.userapi.com/c840729/v840729496/afb4/iiyyVlMojdU.jpg" height="500" width="500">


| Модель               |  roc-auc трэйн | roc-auc тест  | logloss трэйн | logloss тест |
|---------------------:|----------------|---------------|---------------|--------------|
|  Константный (1)     |    0.5         |   0.5         |  0.685        | 14.883       |
|  Лог-регрессия       |    0.683       |   0.670       |  0.631        | 0.638        |
|  Ласс-лог-регр       |    0.683       |   0.670       |  0.631        | 0.638        |
|  Неоптимальное дерево|    1           |   0.608       |  0.000        | 13.283       |
|  Оптимальное дерево  |    0.703       |   0.659       |  0.618        | 0.645        |
|  Базовый лес         |    1           |   0.751       |  0.171        | 0.590        |
|  Оптимальный лес     |    0.709       |   0.682       |  0.623        | 0.632        |
|  XGB                 |    0.991       |   0.763       |  0.300        | 0.572        |



# 4. Двуступенчатая модель

## Датасет номер два. 

Предыдущими моделями нам удалось достичь определённого улучшения в качестве по сравнению с константным прогнозом. Тем не менее, хочется большего! Как уже отмечалось выше, у нас довольно многие люди получали за свою деятельность оценку равную 1. Давайте попробуем построить двухступенчатую модель. 

* Ступень 1: Идентифицируем людей, которые получают единицу. Для этого используем модель, построенную для бинаризованной целевой переменной. Прогнозируем для них единицу.
* Ступень 2: Строим прогнозы для оставщихся людей с помощью модели для непрерывной целевой переменной. 

**Цель: попытаться сделать ошибку меньше 0.38**

Логистическая регрессия  для бинаризованной переменной продемонстрировала себя довольно хорошо. Попробуем использовать её в качестве первой модели. В качестве второй модели попробуем обычную линейную регрессию. 

1. Оцениваем логистическую регрессию на всей выборке. 
2. Оцениваем линейную регрессию на подвыборке, где целевая бинаризованная переменная принимает значение 1 (непрерывная цель больше 1). 
3. Прогнозируем людей, которые получают оценку выше единицы. Для всех таких людей строим прогноз по линейной регрессии. 
4. Считаем среднюю квадратическую ошибку. 

Получаем следующий результат: 

|                   | MSE 1 уровень | MSE 2 уровень | MSE общая     |
|------------------:|---------------|---------------|---------------|
| Трэйн             | 0.039         | 0.055         |   0.051       |             
| Тест              | 0.040         | 0.058         |   0.053       |             

Модель работает хуже, чем все предыдущие. Попробуем на втором шаге оценить линейную регрессию по всей выборке, а не только по подвыборке, на которой целевая переменная больше единицы. Получим более хороший результат. 

|                   | MSE 1 уровень | MSE 2 уровень | MSE общая     |
|------------------:|---------------|---------------|---------------|
| Трэйн             | 0.039         | 0.043         |   0.042       |             
| Тест              | 0.040         | 0.042         |   0.042       |  


Посмотрим более подробно на то как именно работает логистическая регрессия. Точность (precision) её поиска единичек составляет $0.62$. Полнота (recall) составляет $0.42$. Попробуем повысить точность работы первого уровня за счёт понижения полноты. Для этого поставим более высокий порог для прогнозирования единичек. PR-кривая выглядит вот так: 

<img align="center" src="https://sun9-9.userapi.com/c840735/v840735416/c7ed/1Z7Ryq1Jm2U.jpg" height="500" width="500">

Если мы поставим порог на единички высоким, в районе 0.7 - 0.8, то мы получим неплохой прирост в точности. Попробуем сделать это.

| Порог             | 0.5 | 0.7 | 0.8 | 0.85 | 0.9 | 0.95 |
|------------------:|-----|-----|-----|------|-----|------|
| Ошибка   на 1 ур. |0.040|0.021|0.015|0.008 |0.009|0.025 |
| Ошибка   на 2 ур. |0.042|0.039|0.039|0.038 |0.039|0.038 |
| Итоговая ошибка   |0.042|0.038|0.038|0.038 |0.038|0.038 |

В итоге, в результате сдвига порога, нам удалось улучшить результат обычной регрессии на данном датасете с 0.39 до 0.038.  

При этом, пара моделей из двух оптимальных лесов, даёт также качество 0.038. Пара бустингов даёт качество 0.035? что также бьёт предыдущий результат. PR-кривая для бустинга выглядит следующим образом: 

<img align="center" src="https://pp.userapi.com/c840222/v840222571/2c3cd/_ttSIHcHi1M.jpg" height="500" width="500">

Её более выпуклая форма позволяет лучше отбирать единички. Взглянем на распределение прогнозов и на распределение реальных значений $y$. В большей части моделей мы увидим следущую картину.

<img align="center" src="https://sun9-9.userapi.com/c840133/v840133571/29a24/lhCbex2H9h0.jpg" height="500" width="500">

Наша модель не хочет выделять суперуспешных людей. Ни один прогноз не был закинут за точку 1.4. Попробуем решить эту проблему и ещё больше повысить качество с помощью трёхступенчатой модели. 

Будем часть особо успешных людей, которых будет определять дополнительная логистическая регрессия, перекидывать в точку $1.4$. Тогда мы получим следующий результат: 

<img align="center" src="https://sun9-9.userapi.com/c837731/v837731620/669eb/MQ55AqrZ5NY.jpg" height="500" width="500">
